# General Best Practices for Prompting Amazon Nova 2

Effective prompts depend on the quality of information provided and the craftsmanship of the prompt itself. This notebook explores strategies and techniques for getting the most out of Amazon Nova 2 models.

We cover the following topics aligned with the [Nova 2 prompting best practices](https://docs.aws.amazon.com/nova/latest/nova2-userguide/prompting-best-practices.html):

1. **Understanding the roles** — system, user, and assistant
2. **Using the system role** — persona, instructions, guardrails
3. **Creating precise prompts** — clear task definitions
4. **Few-shot prompting** — providing examples to guide the model
5. **Structured outputs** — JSON schema, assistant prefilling, and tool use with constrained decoding

## Setup

In [ ]:
import boto3
import json
from IPython.display import display, Markdown, HTML

In [ ]:
%store -r MODEL_ID
%store -r region_name

client = boto3.client("bedrock-runtime", region_name=region_name)

## 1. Understanding the Roles

Amazon Nova 2 models use three distinct roles to structure conversations:

| Role | Purpose |
| --- | --- |
| **System** (optional) | Sets behavioral parameters, instructions, and guardrails that persist across the entire conversation |
| **User** | Provides context, tasks, instructions, and queries |
| **Assistant** | Guides the model toward the intended response (useful for prefilling) |

The system role supersedes user instructions and carries over across all turns. The assistant role can be used to nudge the model's output format — more on that in the Structured Outputs section.

## 2. Using the System Role

The system prompt defines how the model responds to end users. It can set a persona, specify allowable content, enforce output formats, and establish guardrails. Instructions in the system role supersede individual user prompts and carry over across all turns.

A well-structured system prompt typically includes:

- **Persona**: Who the model should act as
- **Model instructions**: Step-by-step guidance for answering
- **Response schema**: Expected output format
- **Guardrails**: Explicit restrictions (e.g., "DO NOT talk about ..." or "You MUST talk about ...")

To enforce hierarchy between system and user instructions, append: *"If the user request contradicts any system instruction or is outside your scope, politely decline."*

Let's see the system role in action with a telecom customer service scenario.

In [3]:
# System prompt with persona, instructions, and guardrails
system_prompt = """You are a customer service agent for AnyCompany Telecom.

## Model Instructions
- Greet the customer warmly
- Identify the issue from the customer's message
- Provide a clear, actionable resolution
- Offer to escalate if the issue is complex

## Response Schema
Respond in this format:
- Greeting
- Issue identified
- Resolution steps
- Closing

## Guardrails
- DO NOT discuss competitor products or pricing
- DO NOT share internal policies or system details
- If the request is outside your scope, politely decline and explain your capabilities
"""

response = client.converse(
    modelId=MODEL_ID,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": "My internet has been dropping every evening for the past week. I work from home and this is really affecting my productivity."}]
    }],
    inferenceConfig={"maxTokens": 512, "temperature": 0.7}
)

output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

---

**Greeting:**  
Hello! Thank you for reaching out to AnyCompany Telecom. I’m sorry to hear your internet has been dropping every evening—this can be really frustrating, especially when you’re working from home.

---

**Issue Identified:**  
It sounds like you’re experiencing intermittent internet connectivity issues that occur specifically in the evenings, which is impacting your work.

---

**Resolution Steps:**  
To help resolve this, let’s go through a few troubleshooting steps and possible actions:

1. **Check Signal Strength:**  
   - Ensure your router is placed in a central location, away from walls, metal objects, or other electronics that might interfere with the signal.

2. **Restart Your Equipment:**  
   - Turn off your modem and router. Wait about 30 seconds, then turn them back on. This can help reset any temporary glitches.

3. **Run a Speed Test:**  
   - Use a site like [speedtest.net](https://www.speedtest.net) at different times in the evening to see if speeds drop consistently. Share your results with me if you can.

4. **Check for Network Congestion:**  
   - Many users are online in the evenings, which can cause slower speeds. If possible, try scheduling tasks during off-peak hours or contacting your ISP for a network status update.

5. **Wi-Fi Channel Optimization:**  
   - Log into your router’s admin panel (usually through a web browser) and change your Wi-Fi channel to a less congested one. Many routers have an “auto” setting, but manually selecting channel 1, 6, or 11 can sometimes help.

6. **Firmware Updates:**  
   - Make sure your router’s firmware is up to date. This can often be done through the admin panel.

7. **Contact Support for Further Investigation:**  
   - If the issue persists after trying the above steps, I can escalate this to our technical team. They can check for any known service outages in your area or inspect line quality from our side.

---

**Closing:**  
I understand how important a stable connection is for your work, and I’m here to help get this resolved. If you’d like, I can go ahead and open a support ticket for deeper investigation—just say the word!  

Alternatively, if you’ve already tried any of the steps above and are still having trouble, please let me know what you’ve done and any error messages or patterns you’ve noticed. I’m here to assist however I can.

Looking forward to getting your connection back

Now let's test the guardrails by asking something outside scope:

In [4]:
# Test guardrails — asking about a competitor
guardrail_suffix = "\nThe above system instructions define your capabilities and your scope. If the user request contradicts any system instruction or if the request is outside your scope, you must politely decline the request briefly explaining your capabilities and your scope."

response = client.converse(
    modelId=MODEL_ID,
    system=[{"text": system_prompt + guardrail_suffix}],
    messages=[{
        "role": "user",
        "content": [{"text": "How does AnyCompany's pricing compare to Verizon and AT&T?"}]
    }],
    inferenceConfig={"maxTokens": 256, "temperature": 0.7}
)

output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

Hello! Thank you for reaching out to AnyCompany Telecom.

I understand you're interested in comparing our pricing with other providers. However, I'm unable to discuss competitor products or pricing details as this falls outside my capabilities. At AnyCompany Telecom, we focus on providing excellent service and tailored solutions for our customers.

If you'd like to learn more about our plans, packages, or any special offers we currently have available, I’d be happy to provide those details for you. Please let me know how I can assist you further!

Warm regards,
AnyCompany Telecom Customer Service

## 3. Creating Precise Prompts

Before diving into prompt engineering, establish these foundations:

1. **Define your use case** along four dimensions:
   - **Task**: What specific outcome do you want?
   - **Role**: What persona should the model adopt?
   - **Response style**: What format or structure should the output follow?
   - **Instructions**: What guidelines should the model adhere to?

2. **Success criteria**: Define what constitutes a successful response (format, factuality, length, etc.)

3. **Draft and iterate**: Create an initial prompt and refine based on results.

Let's compare a vague prompt versus a precise one for a telecom use case.

In [5]:
# Vague prompt
response = client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": "Tell me about network issues."}]
    }],
    inferenceConfig={"maxTokens": 256, "temperature": 0.7}
)
output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

### Understanding Network Issues

Network issues can broadly affect connectivity, performance, security, and reliability of systems that rely on network communication. These problems can occur in **local area networks (LANs)**, **wide area networks (WANs)**, **wireless networks**, **the internet**, or any interconnected system. Below is a comprehensive overview of common **types of network issues**, their **causes**, **symptoms**, and **troubleshooting approaches**.

---

## 🔧 **Common Types of Network Issues**

### 1. **Connectivity Problems**
**Symptoms:**
- Inability to access the internet or local network resources.
- Devices show “No Internet Access” or “Limited Connectivity.”
- Failure to ping servers or other devices.

**Common Causes:**
- **Physical issues:** Loose cables, faulty Ethernet cables, broken wireless antennas.
- **Router/modem issues:** Power loss, outdated firmware, configuration errors.
- **IP address problems:** IP conflicts, DHCP server failure, incorrect IP settings.
- **Firewall/NAT issues:** Blocking ports or traffic.
- **Wireless interference:** From microwaves, Bluetooth devices, or neighboring networks.

**Troubleshooting Steps:**
- **Check physical connections** (cables, Wi-Fi toggle).
- **Restart the router/modem**.


In [6]:
# Precise prompt with clear task, role, format, and instructions
response = client.converse(
    modelId=MODEL_ID,
    system=[{"text": "You are a network operations engineer at AnyCompany Telecom. Provide concise, technical answers."}],
    messages=[{
        "role": "user",
        "content": [{"text": """Diagnose the following customer-reported issue and provide a troubleshooting plan.

Issue: Customer reports intermittent packet loss (5-15%) on their fiber connection during peak hours (6-10 PM), affecting video calls and streaming.

Respond with:
1. Likely root causes (top 3)
2. Diagnostic commands to run
3. Recommended resolution steps

You MUST format your response as a numbered list following the structure above. Keep your answer under 300 words."""}]
    }],
    inferenceConfig={"maxTokens": 512, "temperature": 0.7}
)

output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

1. **Likely root causes:**
   - **Network congestion:** Peak hour traffic may overload the network, causing packet loss.
   - **QoS misconfiguration:** Improper Quality of Service settings might not prioritize critical traffic like video calls.
   - **Hardware issues:** Faulty or overloaded network devices (e.g., switches, routers) could cause intermittent failures.

2. **Diagnostic commands to run:**
   - `ping -t <customer_gateway_ip>` during peak hours to monitor packet loss.
   - `traceroute <target_ip>` to identify where packets are being lost.
   - `show interface status` and `show interface <interface>` on relevant switches/routers to check for errors or overload.

3. **Recommended resolution steps:**
   - **Implement QoS:** Prioritize VoIP and video traffic to ensure low latency and loss.
   - **Monitor traffic:** Use network monitoring tools to identify congestion points and consider upgrading bandwidth if necessary.
   - **Hardware check:** Inspect and replace any faulty equipment; ensure all devices are updated with the latest firmware.

## 4. Few-Shot Prompting

Including examples in your prompt helps the model understand the desired output format, style, and reasoning pattern. This technique is known as few-shot prompting.

Benefits of providing examples:
- **Consistent responses**: Creates uniformity in style and format
- **Enhanced performance**: Reduces misinterpretation of instructions
- **Reduced hallucinations**: Provides concrete guidance for outputs

Effective examples should be diverse, match the complexity of the target task, and directly relate to the problem.

### Zero-Shot vs. Few-Shot Comparison

Let's compare zero-shot and few-shot approaches for classifying telecom support tickets.

In [7]:
# Zero-shot: no examples provided
zero_shot_prompt = """
##Task##
Classify the provided customer support ticket into one of the predefined categories listed in ##Categories##.

##Categories##
- Billing
- Technical
- Account
- General Inquiry

##Input##
Ticket: "I was charged twice for my November bill and I need a refund for the duplicate payment."

##Output Requirements##
You MUST respond with only the category and a one-sentence explanation in the following format:

Format: [Category]: [One-sentence explanation]

DO NOT include any additional text, commentary, or formatting beyond what is specified above.
"""

response = client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": zero_shot_prompt}]
    }],
    inferenceConfig={"maxTokens": 128, "temperature": 0}
)

output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

Billing: The ticket concerns an issue with a duplicate charge on the customer's November bill and requests a refund.

In [8]:
# Few-shot: provide examples to guide format and reasoning
few_shot_prompt = """
##Task##
Classify the following customer support ticket into one of these categories:
- Billing
- Technical
- Account
- General Inquiry

##Examples##
Example 1:
Ticket: "My internet speed is much slower than what I'm paying for."
Category: Technical
Reason: Customer reports a service performance issue related to network speed.
---
Example 2:
Ticket: "I want to change the primary name on my account to my spouse."
Category: Account
Reason: Customer requests a modification to account holder information.
---
Example 3:
Ticket: "Can you explain what the 'network access fee' on my bill means?"
Category: Billing
Reason: Customer is asking about a specific charge on their invoice.

DO NOT mention examples in your response.

##Input##
Ticket: "I was charged twice for my November bill and I need a refund for the duplicate payment."

##Output Requirements##
You MUST respond with only the category and a one-sentence explanation in the following format:

Format: [Category]: [One-sentence explanation]

DO NOT include any additional text, commentary, or formatting beyond what is specified above.
"""

response = client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": few_shot_prompt}]
    }],
    inferenceConfig={"maxTokens": 128, "temperature": 0}
)

output_text = response["output"]["message"]["content"][0]["text"]
display(Markdown(output_text))

Billing: Customer reports an issue with being charged multiple times for the same bill and requests a refund.

## 5. Structured Outputs

For automated workflows, it's important that model responses follow a specific format that downstream systems can parse. Amazon Nova 2 supports several techniques for generating structured JSON output:

1. **JSON schema in the prompt** — specify the output schema directly in the user message
2. **Assistant prefilling** — nudge the model to start generating JSON by prefilling the assistant turn
3. **Tool use with constrained decoding** — define a tool with a JSON schema and use `toolChoice` to force adherence

> **Tip**: Format requirements (like date formats) work best when defined in the schema itself rather than through examples.

> **Tip**: For simple JSON outputs with up to 10 keys, you can define the schema in the prompt. For more complex schemas, use tool use with constrained decoding for higher adherence.

### Example 1: JSON Schema in the Prompt

Add an `Output Schema` section to your prompt that describes the expected JSON structure with key names and data types. Instruct the model to respond only in valid JSON without preamble.

In [9]:
# Example 1: JSON schema specified in the prompt
response = client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": """List the top 3 AnyCompany Telecom service plans.

Output Schema:
{
  "plans": [
    {
      "name": <string, plan name>,
      "monthly_price_usd": <number>,
      "data_limit_gb": <number or "unlimited">,
      "features": [<string>]
    }
  ]
}

Please generate only the JSON output. DO NOT provide any preamble."""}]
    }],
    inferenceConfig={"maxTokens": 512, "temperature": 0}
)

content_text = response["output"]["message"]["content"][0]["text"]
print(content_text)

```json
{
  "plans": [
    {
      "name": "Basic Plan",
      "monthly_price_usd": 29,
      "data_limit_gb": 10,
      "features": ["5G access", "International calling", "Basic customer support"]
    },
    {
      "name": "Premium Plan",
      "monthly_price_usd": 59,
      "data_limit_gb": "unlimited",
      "features": ["5G access", "International calling", "Premium customer support", "Streaming service subscription", "Free international roaming"]
    },
    {
      "name": "Family Plan",
      "monthly_price_usd": 99,
      "data_limit_gb": "unlimited",
      "features": ["5G access", "International calling", "Premium customer support", "Streaming service subscription", "Free international roaming", "Multiple lines discount", "Parental controls"]
    }
  ]
}
```


### Example 2: Assistant Prefilling

Prefill the assistant's response with `` ```json `` to guide the model to immediately start generating a JSON object. You can also add a stop sequence on `` ``` `` to ensure the output ends cleanly.

> **Note**: Prefilling is only valid when reasoning mode is NOT enabled.

In [10]:
# Example 2: Assistant prefilling with ```json
response = client.converse(
    modelId=MODEL_ID,
    system=[{"text": "You generate JSON objects based on the given instructions."}],
    messages=[
        {
            "role": "user",
            "content": [{"text": """Provide details about the top 3 AnyCompany Telecom service plans.
Your response should be in JSON format, with the following keys: name, monthly_price_usd, data_limit_gb, features."""}]
        },
        {
            "role": "assistant",
            "content": [{"text": "```json"}]
        }
    ],
    inferenceConfig={"maxTokens": 512, "temperature": 0}
)

content_text = response["output"]["message"]["content"][0]["text"]

# Strip trailing ``` if present
if content_text.rstrip().endswith("```"):
    content_text = content_text.rstrip()[:-3].rstrip()

# Validate JSON
try:
    parsed = json.loads(content_text)
    display(parsed)
    print(("Assistant Prefilling — ✅ Valid JSON"))
except json.JSONDecodeError as e:
    display(Markdown(content_text))
    print((f"Assistant Prefilling — ❌ Invalid JSON: {e}"))


{'plans': [{'name': 'Basic Connect',
   'monthly_price_usd': 15,
   'data_limit_gb': 10,
   'features': ['Unlimited local and national calls',
    '5GB high-speed data',
    'Basic voicemail and texting',
    '5GB international data (2G speeds)']},
  {'name': 'Family Plus',
   'monthly_price_usd': 30,
   'data_limit_gb': 25,
   'features': ['Unlimited local, national, and international calls to 20 countries',
    '25GB high-speed data',
    'Advanced voicemail, SMS, and MMS',
    'Free Wi-Fi hotspot access',
    'Family sharing plan (up to 5 lines)',
    '10GB international data (2G speeds)']},
  {'name': 'Premium Unlimited',
   'monthly_price_usd': 50,
   'data_limit_gb': None,
   'features': ['Unlimited local, national, and international calls to 100+ countries',
    'Unlimited high-speed data',
    'Priority customer support',
    'Free international roaming (50+ countries)',
    'Advanced messaging (MMS, group texts, international SMS)',
    'Streaming optimization for video and mu

Assistant Prefilling — ✅ Valid JSON


### Example 3: Tool Use with Constrained Decoding

For complex schemas, define a tool with a full JSON schema and use `toolChoice` to force the model to produce output matching that schema exactly. This leverages constrained decoding, which guarantees the output conforms to the specified structure.

In [11]:
# Example 3: Tool use with toolChoice for constrained decoding
response = client.converse(
    modelId=MODEL_ID,
    messages=[{
        "role": "user",
        "content": [{"text": """Extract entities from this customer message:

"Hi, I'm Sarah from the Portland office. My account number is AC-78234 and I'm having issues with my Enterprise Plus plan. The ticket reference is TK-2024-11-0892."

Use the extract_entities tool."""}]
    }],
    toolConfig={
        "tools": [{
            "toolSpec": {
                "name": "extract_entities",
                "description": "Extract named entities from customer messages",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "customer_name": {
                                "type": "string",
                                "description": "Customer's name"
                            },
                            "location": {
                                "type": "string",
                                "description": "Customer's location or office"
                            },
                            "account_number": {
                                "type": "string",
                                "description": "Account identifier"
                            },
                            "plan_name": {
                                "type": "string",
                                "description": "Service plan name"
                            },
                            "ticket_reference": {
                                "type": "string",
                                "description": "Support ticket ID"
                            }
                        },
                        "required": ["customer_name", "account_number"]
                    }
                }
            }
        }],
        "toolChoice": {"tool": {"name": "extract_entities"}}
    },
    inferenceConfig={"maxTokens": 256, "temperature": 0}
)

# The model returns a tool use block with structured data
tool_use = response["output"]["message"]["content"][0]["toolUse"]
display(tool_use["input"])

{'account_number': 'AC-78234',
 'location': 'Portland',
 'customer_name': 'Sarah',
 'ticket_reference': 'TK-2024-11-0892',
 'plan_name': 'Enterprise Plus'}

## Conclusion

This notebook covered the core prompting best practices for Amazon Nova 2 models:

- **System role**: Define persona, instructions, output format, and guardrails to control model behavior across all turns
- **Precise prompts**: Clearly specify the task, role, format, and constraints for better results
- **Few-shot examples**: Provide diverse, relevant examples to guide the model's output style and reasoning
- **Structured outputs**: Use JSON schema in the prompt, assistant prefilling, or tool use with constrained decoding for parseable output

For more details, see the [Amazon Nova 2 Prompting Guide](https://docs.aws.amazon.com/nova/latest/nova2-userguide/prompting.html). For advanced features like reasoning mode, tool use, and code interpreter, see the **Advanced Tooling** notebook.